# Aggregate round/request results into tables

### 1) Overall configurations 

In [34]:
import os
import os.path
import pandas as pd
from pprint import pprint

# Root
root_path = os.getcwd().replace("\\","/")

# Instance settings
# instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/environment.rebalancing/instance_settings_test_rebalancing.json"
instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/week/allow_hiring.json"

##### Data translation

dict_sq_class = {"A":"Business", "B":"Standard", "C":"Low-cost"}
category_sq_class = pd.api.types.CategoricalDtype(categories=["Business", "Standard", "Low-cost"], ordered=True)

# Vehicle type
dict_service = {"FLEET":"Company-owned", "FREELANCE":"Third-party"}
category_fleet = pd.api.types.CategoricalDtype(categories=["Company-owned", "Third-party"], ordered=True)

# Pickup deadline status (Was user chosen to have its pk deadline extended?)
dict_sl_status = {"MET":"Shortest", "UNMET":"Extended"}
category_status = pd.api.types.CategoricalDtype(categories=["Shortest", "Extended"], ordered=True)

# Establishing category order and alias dictionaries
dict_segmentation = {"AA":"B+", "BB":"S+", "CC":"L+", "A":"B", "B":"S", "C":"L"}
category_segmentation = pd.api.types.CategoricalDtype(categories=["B+", "S+", "L+", "B", "S", "L"], ordered=True)

dict_contract_duration = {"3600":"1h", "0":"Single-ride", "10800":"3h", "-1":"-",  "BASELINE":"Baseline"}
category_contract_duration = pd.api.types.CategoricalDtype(categories=["Single-ride", "1h", "3h"], ordered=True)

dict_service_rate = {"S1":"SR1", "S2":"SR2", "S3":"SR3", "BASELINE":"Baseline"}
category_service_rate = pd.api.types.CategoricalDtype(categories=["SR1", "SR2", "SR3"], ordered=True)


### 2) Loading the instance data

In [2]:
import json

def load_json(path):
    """Read json file and return dictionary"""

    # Add .json to the end of file if needed
    if path.find(".json") < 0:
        path = path + ".json"

    # Read JSON file
    with open(path) as data_file:
        data_loaded = json.load(data_file)

    return data_loaded

instances_dic = load_json(instance_settings_path)

# Folder where results will be saved
result_folder = instances_dic["result_folder"]

print("########### INSTANCE SETTINGS ##################################################")
pprint(instances_dic)

########### INSTANCE SETTINGS ##################################################
{'instance_description': 'Whole week experiment (hiring and deteriorating '
                         'enabled)',
 'instance_name': 'HIRINGWEEKMAXWAITINGREB2',
 'instances_folder': 'C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/week/',
 'labels': {'BA': 'batch_duration',
            'CD': 'contract_duration',
            'CS': 'customer_segmentation',
            'CT': 'clear_target_list_every_round',
            'ID': 'instance_description',
            'IF': 'initial_fleet',
            'IN': 'instance_name',
            'MC': 'max_capacity',
            'MO': 'allow_many_to_one',
            'MR': 'max_requests',
            'RE': 'rebalance',
            'RT': 'reinsert_targets',
            'SD': 'allow_service_deterioration',
            'SR': 'service_rate',
            'ST': 'simulation_time',
            'UR': 'allow_urgent_relocation',
            'VH': 'allow_vehicle_hiring'},
 'rebala

### Get settings from instance name

E.g.:

* Input = `IN-instanceName_BA-30_TH-86400_MR-1000_IF-1000_MC-06_CD-3600-SR-S1_CS-AA_SD_VH_MO_RT_CT_UR`

* Output = 
{allow_many_to_one: True,
allow_service_deterioration: True,
allow_urgent_relocation: True,
allow_vehicle_hiring: True,
batch_duration: 30,
clear_target_list_every_round: True,
contract_duration: 3600,
customer_segmentation: AA,
initial_fleet: 1000,
max_capacity: 06,
max_requests: 1000,
reinsert_targets: True,
time_horizon: 86400}

In [3]:
def get_instance_settings(file_name):
    """ Read file name and return instance settings.
    E.g.:
     Input = IN-instanceName_BA-30_TH-86400_MR-1000_IF-1000_MC-06_CD-3600-SR-S1_CS-AA_SD_VH_MO_RT_CT_UR
     Output = {'allow_many_to_one': True,
                'allow_service_deterioration': True,
                'allow_urgent_relocation': True,
                'allow_vehicle_hiring': True,
                'batch_duration': '30',
                'clear_target_list_every_round': True,
                'contract_duration': '3600',
                'customer_segmentation': 'AA',
                'initial_fleet': '1000',
                'max_capacity': '06',
                'max_requests': '1000',
                'reinsert_targets': True,
                'time_horizon': '86400'}
    """
    label_setting_dic = instances_dic["labels"]
    
    # print(file_name)
        
    # E.g., ['BA-30', 'TH-86400', 'MR-1000', 'IF-1000', 'MC-06', 'CD-3600-SR-S1', 'CS-AA', 'SD', 'VH', 'MO', 'RT', 'CT', 'UR']
    file_instances = file_name.split("_")

    instance_settings = dict()

    for e in file_instances:
        
        if e in label_setting_dic.keys():
            # E.g., e =  SD
            k = label_setting_dic[e]
            # E.g., k = allow_service_deterioration
            instance_settings[k] = True
            
        else:
            # E.g., lv = ["BA", "30"]
            lv  = e.split('-')
            # E.g., e2 = BA
            e2 = lv[0]
            # E.g., k = batch_duration
            k2 = label_setting_dic[e2]
            
            if len(lv) > 1:
                 # E.g., v = '30'
                v = lv[1]
                instance_settings[k2] = v
            else:
                # label is not in instance name = False
                instance_settings[k] = False

    return instance_settings

### Aggregate results (folder request_track)

* `earliest`
* `id` = 1, 2, 3, ..., #USERS
* `class` = A, B, C
* `pk_delay`
* `ride_delay`
* `pk_time`
* `dp_time`
* `id_from` = Network id
* `id_to` = Network id
* `dist` = trip(id_from, id_to) in seconds
* `service` = {FLEET, FREELANCE}
* `service_level` = {MET, UNMET}

In [4]:
import pandas as pd
from pprint import pprint

def get_request_track_dic(path_experiment, name_experiment):
    
    # Load results
    experiment_file = "{}request_track/{}.csv".format(path_experiment, name_experiment)
    
    # print("Processing experiment file '{}'".format(experiment_file))
    df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)
    
    aggfunc = {"pk_delay" : ['mean', 'count', 'max']}
    
    dfp = df.pivot_table(index="class", columns="service_level", aggfunc=aggfunc, values=["pk_delay"])

    return dfp

### Processing all instances in folder (mean time)

In [61]:
from collections import defaultdict
import math


request_log_folder = result_folder + "request_track/"

# Get all instances in folder
instance_file_names = os.listdir(request_log_folder)
print("Reading files in folder:", request_log_folder)


instance_attribute = defaultdict(list)
count_lim = 0
for file_name in instance_file_names:
    #if count_lim > 2:
    #    break
    count_lim = count_lim + 1
    instance, extension = file_name.split(".")
    
    
    # Instance settings
    instance_settings = get_instance_settings(instance)
        
    # Filter fixed fleet instances
    #if instance_settings.get('service_rate', '-') == '-':
    #    continue
     
    # Filter instance label
    if instance_settings['instance_name'] == "HIRINGWEEKMAXWAITINGREB2" or 'rebalance' not in instance_settings.keys():
        continue
    
    
    # Filter user base
    if instance_settings['customer_segmentation'] in ['A', 'B', 'C']:
        continue
    
    # Filter service rate
    if instance_settings.get('service_rate', None) in ['S1']:
        continue
        
    print("  - Processing", instance)
    
    # Load results
    experiment_file = "{}{}.csv".format(request_log_folder, instance)
    df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)

    column = ('', 'contract_duration')
    # When no contract duration is available, instance refers to baseline
    instance_attribute[column].append(int(instance_settings.get('contract_duration', '-1')))

    column = ('', 'customer_segmentation')
    instance_attribute[column].append(instance_settings['customer_segmentation'])

   
    # Convert denied to freelance to show denied under same header of the initial fleet
    df.loc[(df['service']=='DENIED'), 'service'] = 'FLEET'
    
    
    for sq_class in ['A', 'B', 'C']:
        for sl_status in ["MET", "UNMET"]:
            # Categories leading to max, mean and count
            super_indexes = (dict_sq_class[sq_class], dict_sl_status[sl_status])

            # Filter dataframe
            filter_sq_sl = (df['class'] == sq_class) & (df['service_level'] == sl_status)
            df_filtered = df.loc[filter_sq_sl]["pk_delay"]
            
            mean_pk = df_filtered.mean()

            # Format data
            mean_pk_min = (mean_pk if not math.isnan(mean_pk) else 0)
            instance_attribute[super_indexes].append(mean_pk_min)

Reading files in folder: C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/week/request_track/
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S2_VH_SD_RE_CT_RT_UR
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_CD-10800_SR-S2_VH_SD_RE_CT_RT_UR
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_CD-3600_SR-S2_VH_SD_RE_CT_RT_UR
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-BB_CD-0_SR-S2_VH_SD_RE_CT_RT_UR
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-BB_CD-10800_SR-S2_VH_SD_RE_CT_RT_UR
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-BB_CD-3600_SR-S2_VH_SD_RE_CT_RT_UR
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-CC_CD-0_SR-S2_VH_SD_RE_CT_RT_UR
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-CC_CD-10800_SR-S2_VH_SD_RE_C

In [63]:
from datetime import datetime, timedelta
#pprint(instance_attribute)

# Build data frame from dictionary
a = pd.DataFrame.from_dict(instance_attribute)
a

Business              \
   contract_duration customer_segmentation    Shortest    Extended   
0                  0                    AA  122.487745  288.993312   
1              10800                    AA  101.948326  234.144420   
2               3600                    AA  116.929324  242.125293   
3                  0                    BB  113.244624  276.645719   
4              10800                    BB  106.783355  279.367347   
5               3600                    BB  111.211065  290.019695   
6                  0                    CC  113.154254  258.610453   
7              10800                    CC  108.244051  287.620335   
8               3600                    CC  111.872398  288.815463   
9                 -1                    AA  241.748524    0.000000   
10                -1                    BB  167.183901    0.000000   
11                -1                    CC  165.546919    0.000000   
12                -1                    AA  123.239303    0.000000   
13                -1                    BB  110.820065    0.000000   
14                -1                    CC  111.099878    0.000000   

      Standard                Low-cost               
      Shortest    Extended    Shortest     Extended  
0   158.830147  456.465116  211.391242   695.000000  
1   109.065321  368.400000  110.579184     0.000000  
2   134.489886  360.000000  138.002247     0.000000  
3   119.011018  498.888136  161.425019     0.000000  
4   104.544675  401.701149  118.215166     0.000000  
5   112.022361  429.650327  135.595006     0.000000  
6   126.880935  490.711436  146.162719  1111.560748  
7   113.574728  443.240000  114.641835   782.000000  
8   120.163077  453.287356  127.511524   762.500000  
9   297.019644    0.000000  438.349683     0.000000  
10  164.291436    0.000000  216.728653     0.000000  
11  179.195204    0.000000  208.785219     0.000000  
12  158.939236    0.000000  206.584574     0.000000  
13  117.724317    0.000000  158.171116     0.000000  
14  125.343607    0.000000  143.307304     0.000000

### Table: What is the service level (pickup delay) of the users lying outside SQ-class service rate?

In [64]:

key_cs = ('', 'customer_segmentation') # (A, AA, BB, etc.)
key_cd = ('',  'contract_duration') #(0, 3600, 18000)

# Filtering data (only mixed segmentations scenarios will be used)
a = a[a[key_cs].isin(["AA", "BB", "CC"])]

# Renaming data and applying aliases
a[key_cs] = a[key_cs].map(lambda e:dict_segmentation[e])
a[key_cs] = a[key_cs].astype(category_segmentation)

a[key_cd] = a[key_cd].map(lambda e:dict_contract_duration[str(e)])
a[key_cd] = a[key_cd].astype(category_contract_duration)

a = a.sort_values(by=[key_cs,key_cd])
a = a.set_index([key_cs,key_cd])

# Changing table column index names
a.index.names = ["User base", "Contract duration"]

# Transforming to minutes
#a = a.applymap(lambda e: ('{:.2f}'.format(float(e)/60) if e != '-' else e))
a = a.applymap(lambda e: ('{:02}:{:02}'.format(int(e)//60, int(e)%60) if e != 0 else '-'))
a

Business          Standard          Low-cost  \
                            Shortest Extended Shortest Extended Shortest   
User base Contract duration                                                
B+        Single-ride          02:02    04:48    02:38    07:36    03:31   
          1h                   01:56    04:02    02:14    06:00    02:18   
          3h                   01:41    03:54    01:49    06:08    01:50   
          NaN                  04:01        -    04:57        -    07:18   
          NaN                  02:03        -    02:38        -    03:26   
S+        Single-ride          01:53    04:36    01:59    08:18    02:41   
          1h                   01:51    04:50    01:52    07:09    02:15   
          3h                   01:46    04:39    01:44    06:41    01:58   
          NaN                  02:47        -    02:44        -    03:36   
          NaN                  01:50        -    01:57        -    02:38   
L+        Single-ride          01:53    04:18    02:06    08:10    02:26   
          1h                   01:51    04:48    02:00    07:33    02:07   
          3h                   01:48    04:47    01:53    07:23    01:54   
          NaN                  02:45        -    02:59        -    03:28   
          NaN                  01:51        -    02:05        -    02:23   

                                      
                            Extended  
User base Contract duration           
B+        Single-ride          11:35  
          1h                       -  
          3h                       -  
          NaN                      -  
          NaN                      -  
S+        Single-ride              -  
          1h                       -  
          3h                       -  
          NaN                      -  
          NaN                      -  
L+        Single-ride          18:31  
          1h                   12:42  
          3h                   13:02  
          NaN                      -  
          NaN                      -

### Getting latex table (median delay)

In [65]:
print(a.to_latex(multicolumn=True, multirow=True))

\begin{tabular}{llllllll}
\toprule
   &     & \multicolumn{2}{l}{Business} & \multicolumn{2}{l}{Standard} & \multicolumn{2}{l}{Low-cost} \\
   &     & Shortest & Extended & Shortest & Extended & Shortest & Extended \\
User base & Contract duration &          &          &          &          &          &          \\
\midrule
\multirow{5}{*}{B+} & Single-ride &    02:02 &    04:48 &    02:38 &    07:36 &    03:31 &    11:35 \\
   & 1h &    01:56 &    04:02 &    02:14 &    06:00 &    02:18 &        - \\
   & 3h &    01:41 &    03:54 &    01:49 &    06:08 &    01:50 &        - \\
   & NaN &    04:01 &        - &    04:57 &        - &    07:18 &        - \\
   & NaN &    02:03 &        - &    02:38 &        - &    03:26 &        - \\
\cline{1-8}
\multirow{5}{*}{S+} & Single-ride &    01:53 &    04:36 &    01:59 &    08:18 &    02:41 &        - \\
   & 1h &    01:51 &    04:50 &    01:52 &    07:09 &    02:15 &        - \\
   & 3h &    01:46 &    04:39 &    01:44 &    06:41 &    01:58 &     

### Processing all instances (count)

In [15]:
from collections import defaultdict
import math


request_log_folder = result_folder + "request_track/"

# Get all instances in folder
instance_file_names = os.listdir(request_log_folder)
print("Reading files in folder:", request_log_folder)

sl_status = "MET"

instance_attribute_count = defaultdict(list)
instance_attribute_count_percentage = defaultdict(list)
count_lim = 0
for file_name in instance_file_names:
    #if count_lim > 0:
    #    break
    count_lim = count_lim + 1
    instance, extension = file_name.split(".")
    
    
    # Instance settings
    instance_settings = get_instance_settings(instance)
        
    # Filter fixed fleet instances
    #if instance_settings.get('service_rate', '-') == '-':
    #    continue
        
    if instance_settings['instance_name'] == "HIRINGWEEKMAXWAITINGREB2" or 'rebalance' not in instance_settings.keys():
        continue
    
    if instance_settings['customer_segmentation'] in ['A', 'B', 'C']:
        continue
    
    # Filter service rate
    if instance_settings.get('service_rate', None) in ['S1']:
        continue
        
    #if instance_settings['instance_name'] != "WEEKDENY" or 'rebalance' not in instance_settings.keys():
    #    continue
    
    print("  - Processing", instance)
    
    try:
        # Load results
        experiment_file = "{}{}.csv".format(request_log_folder, instance)
        df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)
    
        column = ('', 'contract_duration')
        # When no contract duration is available, instance refers to baseline
        instance_attribute_count[column].append(instance_settings.get('contract_duration', 'BASELINE'))
        instance_attribute_count_percentage[column].append(instance_settings.get('contract_duration', 'BASELINE'))
            
        column = ('', 'customer_segmentation')
        instance_attribute_count[column].append(instance_settings['customer_segmentation'])
        instance_attribute_count_percentage[column].append(instance_settings['customer_segmentation'])
        
        # Get total number of requests
        total = len(df)
        
        # Convert denied to freelance to show denied under same header of the initial fleet
        df.loc[(df['service']=='DENIED'), 'service'] = 'FLEET'
        
        print(df.service.value_counts())
        
        for sq_class in ['A', 'B', 'C']:
            for fleet in ['FLEET', 'FREELANCE']:

                # Categories leading to max, mean and count
                #super_indexes = (dict_service[fleet], dict_sq_class[sq_class], dict_sl_status[sl_status])
                #super_indexes = (dict_service[fleet], dict_sl_status[sl_status], dict_sq_class[sq_class])
                super_indexes = (dict_sq_class[sq_class], dict_service[fleet])

                # Filter dataframe
                filter_sq_sl = (df['service'] == fleet) & (df['class'] == sq_class) & (df['service_level'] == sl_status)
                df_filtered = df.loc[filter_sq_sl]["pk_delay"]

                mean_pk = df_filtered.mean()
                max_pk = df_filtered.max()

                count = df_filtered.count()
                instance_attribute_count[super_indexes].append(count)

                count = (count/total if not (math.isnan(count) or count==0) else 0)
                instance_attribute_count_percentage[super_indexes].append(count)

                #instance_attribute[(*super_indexes, 'Max.')].append(max_pk_min)
                # instance_attribute[(*super_indexes, 'Mean.')].append(mean_pk_min)

                print(instance_attribute_count_percentage)
        for fleet in ['FLEET', 'FREELANCE']:

            filter_sq_sl = (df['service'] == fleet) & (df['service_level'] == sl_status)
            subtotal_index = ("Total", dict_service[fleet])
            
            count_fleet = len(df.loc[filter_sq_sl])
            instance_attribute_count[subtotal_index].append(count_fleet)

            count_fleet = (count_fleet/total if not (math.isnan(count_fleet) or count_fleet==0) else 0)
            instance_attribute_count_percentage[subtotal_index].append(count_fleet)
            
    except Exception as e:
        print(e)
        pass

Reading files in folder: C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/week/request_track/
  - Processing IN-HIRINGWEEKMAXWAITING_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S2_VH_SD_RE_CT_RT_UR
FLEET        1411447
FREELANCE     173625
Name: service, dtype: int64
defaultdict(<class 'list'>, {('', 'contract_duration'): ['0'], ('', 'customer_segmentation'): ['AA'], ('Business', 'Company-owned'): [0.5584883996025425]})
defaultdict(<class 'list'>, {('', 'contract_duration'): ['0'], ('', 'customer_segmentation'): ['AA'], ('Business', 'Company-owned'): [0.5584883996025425], ('Business', 'Third-party'): [0.10937652792454616]})
defaultdict(<class 'list'>, {('', 'contract_duration'): ['0'], ('', 'customer_segmentation'): ['AA'], ('Business', 'Company-owned'): [0.5584883996025425], ('Business', 'Third-party'): [0.10937652792454616], ('Standard', 'Company-owned'): [0.15987634654511615]})
defaultdict(<class 'list'>, {('', 'contract_duration'): ['0'], ('', 'customer_segmentation'

In [16]:
instance_attribute_count_percentage

defaultdict(list,
            {('', 'contract_duration'): ['0',
              '10800',
              '3600',
              '0',
              '10800',
              '3600',
              '0',
              '10800',
              '3600',
              'BASELINE',
              'BASELINE',
              'BASELINE',
              'BASELINE',
              'BASELINE',
              'BASELINE'],
             ('', 'customer_segmentation'): ['AA',
              'AA',
              'AA',
              'BB',
              'BB',
              'BB',
              'CC',
              'CC',
              'CC',
              'AA',
              'BB',
              'CC',
              'AA',
              'BB',
              'CC'],
             ('Business', 'Company-owned'): [0.5584883996025425,
              0.49336340551708907,
              0.5281996120057411,
              0.13464914909389145,
              0.1373039130640506,
              0.1419036954087346,
              0.13697711464757187,
  

### Table: How many users were serviced by each vehicle type?

In [39]:
# Build data frame from dictionary
b = pd.DataFrame.from_dict(instance_attribute_count_percentage)
key_cs = ('',  'customer_segmentation') # (A, AA, BB, etc.)
key_cd = ('',  'contract_duration') #(0, 3600, 18000)

# Renaming data and applying aliases
b[key_cs] = b[key_cs].map(lambda e:dict_segmentation[e])
b[key_cs] = b[key_cs].astype(category_segmentation)

b[key_cd] = b[key_cd].map(lambda e:dict_contract_duration[e])
b[key_cd] = b[key_cd].astype(category_contract_duration)

b = b.sort_values(by=[key_cs,key_cd])
b = b.set_index([key_cs,key_cd])
b.index.names = ["User base", "Contract duration"]


def formatResult(e):
    if e == 0:
        return '-'
    if e < 0.01:
        return '*'
    return '{:.2%}'.format(float(e))

overall_total = b.copy()
# Transforming to minutes
b = b.applymap(lambda e: formatResult(e))
b

Business                  Standard  \
                            Company-owned Third-party Company-owned   
User base Contract duration                                           
B+        Single-ride              55.85%      10.94%        15.99%   
          1h                       52.82%      14.86%        12.56%   
          3h                       49.34%      18.52%        11.90%   
          NaN                      59.43%           -        16.00%   
          NaN                      57.33%           -        15.99%   
S+        Single-ride              13.46%       2.27%        67.34%   
          1h                       14.19%       1.71%        62.93%   
          3h                       13.73%       2.21%        59.78%   
          NaN                      14.78%           -        67.86%   
          NaN                      13.68%           -        67.49%   
L+        Single-ride              13.70%       2.06%        15.75%   
          1h                       14.53%       1.38%        15.19%   
          3h                       14.13%       1.82%        14.49%   
          NaN                      14.67%           -        15.82%   
          NaN                      13.87%           -        15.80%   

                                             Low-cost              \
                            Third-party Company-owned Third-party   
User base Contract duration                                         
B+        Single-ride                 *        15.99%           *   
          1h                      3.43%        12.59%       3.42%   
          3h                      4.10%        11.89%       4.11%   
          NaN                         -        16.00%           -   
          NaN                         -        16.00%           -   
S+        Single-ride                 *        16.01%           *   
          1h                      5.05%        14.57%       1.43%   
          3h                      8.22%        14.02%       1.99%   
          NaN                         -        16.00%           -   
          NaN                         -        16.00%           -   
L+        Single-ride                 *        67.98%           *   
          1h                          *        63.52%       4.49%   
          3h                      1.51%        60.76%       7.25%   
          NaN                         -        68.00%           -   
          NaN                         -        68.00%           -   

                                    Total              
                            Company-owned Third-party  
User base Contract duration                            
B+        Single-ride              87.83%      10.95%  
          1h                       77.97%      21.70%  
          3h                       73.13%      26.73%  
          NaN                      91.43%           -  
          NaN                      89.32%           -  
S+        Single-ride              96.81%       2.80%  
          1h                       91.69%       8.19%  
          3h                       87.53%      12.41%  
          NaN                      98.63%           -  
          NaN                      97.18%           -  
L+        Single-ride              97.43%       2.28%  
          1h                       93.24%       6.66%  
          3h                       89.37%      10.58%  
          NaN                      98.49%           -  
          NaN                      97.67%           -

In [44]:
overall_total[("", "Total")]=overall_total[("Total", 'Company-owned')] + overall_total[("Total", 'Third-party')]
overall_total = overall_total.applymap(lambda e: formatResult(e))
overall_total

Business                  Standard  \
                            Company-owned Third-party Company-owned   
User base Contract duration                                           
B+        Single-ride              55.85%      10.94%        15.99%   
          1h                       52.82%      14.86%        12.56%   
          3h                       49.34%      18.52%        11.90%   
          NaN                      59.43%           -        16.00%   
          NaN                      57.33%           -        15.99%   
S+        Single-ride              13.46%       2.27%        67.34%   
          1h                       14.19%       1.71%        62.93%   
          3h                       13.73%       2.21%        59.78%   
          NaN                      14.78%           -        67.86%   
          NaN                      13.68%           -        67.49%   
L+        Single-ride              13.70%       2.06%        15.75%   
          1h                       14.53%       1.38%        15.19%   
          3h                       14.13%       1.82%        14.49%   
          NaN                      14.67%           -        15.82%   
          NaN                      13.87%           -        15.80%   

                                             Low-cost              \
                            Third-party Company-owned Third-party   
User base Contract duration                                         
B+        Single-ride                 *        15.99%           *   
          1h                      3.43%        12.59%       3.42%   
          3h                      4.10%        11.89%       4.11%   
          NaN                         -        16.00%           -   
          NaN                         -        16.00%           -   
S+        Single-ride                 *        16.01%           *   
          1h                      5.05%        14.57%       1.43%   
          3h                      8.22%        14.02%       1.99%   
          NaN                         -        16.00%           -   
          NaN                         -        16.00%           -   
L+        Single-ride                 *        67.98%           *   
          1h                          *        63.52%       4.49%   
          3h                      1.51%        60.76%       7.25%   
          NaN                         -        68.00%           -   
          NaN                         -        68.00%           -   

                                    Total                      
                            Company-owned Third-party   Total  
User base Contract duration                                    
B+        Single-ride              87.83%      10.95%  98.78%  
          1h                       77.97%      21.70%  99.68%  
          3h                       73.13%      26.73%  99.86%  
          NaN                      91.43%           -  91.43%  
          NaN                      89.32%           -  89.32%  
S+        Single-ride              96.81%       2.80%  99.62%  
          1h                       91.69%       8.19%  99.88%  
          3h                       87.53%      12.41%  99.94%  
          NaN                      98.63%           -  98.63%  
          NaN                      97.18%           -  97.18%  
L+        Single-ride              97.43%       2.28%  99.71%  
          1h                       93.24%       6.66%  99.91%  
          3h                       89.37%      10.58%  99.95%  
          NaN                      98.49%           -  98.49%  
          NaN                      97.67%           -  97.67%

In [45]:
print(overall_total.to_latex(multicolumn=True, multirow=True))

\begin{tabular}{lllllllllll}
\toprule
   &     & \multicolumn{2}{l}{Business} & \multicolumn{2}{l}{Standard} & \multicolumn{2}{l}{Low-cost} & \multicolumn{3}{l}{Total} \\
   &     & Company-owned & Third-party & Company-owned & Third-party & Company-owned & Third-party & Company-owned & Third-party &   Total \\
User base & Contract duration &               &             &               &             &               &             &               &             &         \\
\midrule
\multirow{5}{*}{B+} & Single-ride &        55.85\% &      10.94\% &        15.99\% &           * &        15.99\% &           * &        87.83\% &      10.95\% &  98.78\% \\
   & 1h &        52.82\% &      14.86\% &        12.56\% &       3.43\% &        12.59\% &       3.42\% &        77.97\% &      21.70\% &  99.68\% \\
   & 3h &        49.34\% &      18.52\% &        11.90\% &       4.10\% &        11.89\% &       4.11\% &        73.13\% &      26.73\% &  99.86\% \\
   & NaN &        59.43\% &           - &

In [ ]:
b.to_csv("Counts.csv")

In [ ]:
file = "IN-WEEKDENY_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_RE_CT_RT_UR.csv"

user = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True, comment="#")

In [ ]:
user.service_level.value_counts()

In [ ]:
if 'Company-ownedd' in ('Low-cost', 'Company-owned', 'Extended'):
    print("e")

In [ ]:
c = pd.DataFrame(b)

In [ ]:
extended[[(a, b, c) for a in ['Business', 'Standard', 'Low-cost'] for b in ['Company-owned', 'Third-party'] for c in ['Extended']]]